In [1]:
import pandas as pd
import numpy as np

In [2]:
entprise_train = pd.read_csv('train/entprise_info.csv',engine='python')
base_info = pd.read_csv('train/base_info.csv',engine='python',encoding='utf-8')
entprise_test = pd.read_csv('entprise_evaluate.csv')

In [3]:
entprise = pd.concat([entprise_train,entprise_test])
data = pd.merge(entprise,base_info,on='id')
data.drop(['ptbusscope','midpreindcode','score'],axis=1,inplace=True)

In [4]:
data['enttypeitem'] = data['enttypegb']//10*10
data['enttypeminu'] = data['enttypegb']

In [5]:
data['opform'].fillna(10,inplace=True)
data['opform'] = data['opform'].astype(str)

In [6]:
from sklearn.preprocessing import LabelEncoder
labelencoder1 = LabelEncoder()
labelencoder2 = LabelEncoder()
labelencoder3 = LabelEncoder()
labelencoder4 = LabelEncoder()
data['opform'] = labelencoder1.fit_transform(data['opform'])
data['oploc'] = labelencoder2.fit_transform(data['oploc'])
data['industryphy'] = labelencoder3.fit_transform(data['industryphy'])
data['oplocdistrict'] = labelencoder4.fit_transform(data['oplocdistrict'])

In [7]:
def opto_process(x):
    if len(x)>3:
        return x[:4]
    else:
        return x
data['opfrom'] = data['opfrom'].map(lambda x:x[:4])
data['opto'].fillna('nan',inplace=True)
data['opto'] = data['opto'].map(opto_process)
data['opfrom'] = data['opfrom'].astype(int)
data['opto'] = data['opto'].astype(float)

In [8]:
data['industryco'] = data['industryco'].fillna(data['industryco'].mean())

In [9]:
data.loc[data['opto'].isnull(),"opto"] = data[data['opto'].isnull()]['opfrom']+50.0

In [10]:
data['opscope'] = data['opscope'].apply(lambda x:x.replace("、","/").replace("；","/").replace("，","/").replace("。","/"))
data['opscope'] = data['opscope'].apply(lambda x:x.replace("（","/").replace("）","/").replace("，","/").replace("。","/"))
data['opscope'] = data['opscope'].apply(lambda x:"".join(x.split("/")))
import jieba
data['opscope_1'] = data['opscope'].apply(lambda x:jieba.lcut(x,cut_all=True))
from collections import Counter
def jieba_func(x):
    str1 = list(Counter(x).keys())[0]
    str2 = list(Counter(x).keys())[1]
    count = str1+str2
    return count
data['count_opscope'] = data['opscope'].apply(lambda x:jieba_func(x))
le6 = LabelEncoder()
data['opscope_length'] = data['opscope_1'].apply(lambda x:len(x))
# data['opscope'] = data['opscope'].apply(lambda x:x.split('/')[-2:])
data['opscope_label'] = le6.fit_transform(data['count_opscope'])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.540 seconds.
Prefix dict has been built successfully.


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24865 entries, 0 to 24864
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              24865 non-null  object 
 1   label           14865 non-null  float64
 2   oplocdistrict   24865 non-null  int64  
 3   industryphy     24865 non-null  int64  
 4   industryco      24865 non-null  float64
 5   dom             24865 non-null  object 
 6   opscope         24865 non-null  object 
 7   enttype         24865 non-null  int64  
 8   enttypeitem     24865 non-null  int64  
 9   opfrom          24865 non-null  int32  
 10  opto            24865 non-null  float64
 11  state           24865 non-null  int64  
 12  orgid           24865 non-null  int64  
 13  jobid           24865 non-null  int64  
 14  adbusign        24865 non-null  int64  
 15  townsign        24865 non-null  int64  
 16  regtype         24865 non-null  int64  
 17  empnum          19615 non-null 

In [12]:
columns = data.columns.tolist()
for i in columns:
    count_null = data[i].isnull().sum()
    avg_null = count_null/data.shape[0]
    if avg_null>0.5:
        if i == 'reccap':
            continue
        else:
            data.drop(i,axis=1,inplace=True)
    else:
        continue

In [13]:
columns = data.columns.tolist()
for j in columns:
    columns_dtype = data[j].dtype
    if columns_dtype == 'object':
        data.drop(j,axis=1,inplace=True)
    else:
        continue

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24865 entries, 0 to 24864
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   label           14865 non-null  float64
 1   oplocdistrict   24865 non-null  int64  
 2   industryphy     24865 non-null  int64  
 3   industryco      24865 non-null  float64
 4   enttype         24865 non-null  int64  
 5   enttypeitem     24865 non-null  int64  
 6   opfrom          24865 non-null  int32  
 7   opto            24865 non-null  float64
 8   state           24865 non-null  int64  
 9   orgid           24865 non-null  int64  
 10  jobid           24865 non-null  int64  
 11  adbusign        24865 non-null  int64  
 12  townsign        24865 non-null  int64  
 13  regtype         24865 non-null  int64  
 14  empnum          19615 non-null  float64
 15  opform          24865 non-null  int64  
 16  enttypeminu     24865 non-null  int64  
 17  oploc           24865 non-null 

In [33]:
x_train = data[data['reccap'].notnull()]
y_train = x_train['reccap']
x_train = x_train.drop("reccap",axis=1)
x_test = data[data['reccap'].isnull()]
x_test = x_test.drop('reccap',axis=1)

In [34]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7084 entries, 0 to 24861
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   label           2982 non-null   float64
 1   oplocdistrict   7084 non-null   int64  
 2   industryphy     7084 non-null   int64  
 3   industryco      7084 non-null   float64
 4   enttype         7084 non-null   int64  
 5   enttypeitem     7084 non-null   int64  
 6   opfrom          7084 non-null   int32  
 7   opto            7084 non-null   float64
 8   state           7084 non-null   int64  
 9   orgid           7084 non-null   int64  
 10  jobid           7084 non-null   int64  
 11  adbusign        7084 non-null   int64  
 12  townsign        7084 non-null   int64  
 13  regtype         7084 non-null   int64  
 14  empnum          5161 non-null   float64
 15  opform          7084 non-null   int64  
 16  enttypeminu     7084 non-null   int64  
 17  oploc           7084 non-null   

In [42]:
from sklearn.metrics import auc, roc_curve,f1_score,precision_score,recall_score,mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestClassifier
# 模型
def makecatr():
    params = {
        "learning_rate":0.1,
        'iterations':10000,
        'depth': 5,
        'l2_leaf_reg': 3,
        'bootstrap_type':'Bernoulli',
        'od_type': 'Iter',
        'od_wait': 50,
        'random_seed': 2020,
        'allow_writing_files': False,
        'task_type':'CPU',
        'verbose':500,
        'loss_function':'RMSE',
    }
    
    catr = CatBoostRegressor(**params)
    return catr
# 本地验证
kf = KFold(n_splits=5,shuffle=True,random_state=2020)
devscore = []
tevscore = []
count = 1
for tidx,didx in kf.split(x_train.index):
    print('************************************ {} ************************************'.format(str(count)))
    count += 1
    tf = x_train.iloc[tidx]
    df = x_train.iloc[didx]
    tt = y_train.iloc[tidx]
    dt = y_train.iloc[didx]
    catr = makecatr()
    catr.fit(tf, tt)
    pre_train = catr.predict(tf)
    score = mean_squared_error(pre_train,tt)
    tevscore.append(score)
    print("train F1：%s"%score)
    pre = catr.predict(df)
    score = mean_squared_error(pre,dt)
    devscore.append(score)
    print("Valid F1：%s"%score)
print("Train average F1：%s"%np.mean(tevscore),"Valid average F1：%s"%np.mean(devscore))
# pre = rf.predict(x_test)

************************************ 1 ************************************
0:	learn: 37712.2798939	total: 1.69ms	remaining: 16.9s
500:	learn: 6191.7279968	total: 685ms	remaining: 13s
1000:	learn: 4562.0703726	total: 1.39s	remaining: 12.5s
1500:	learn: 3915.5190329	total: 2.08s	remaining: 11.8s
2000:	learn: 3605.3904685	total: 2.78s	remaining: 11.1s
2500:	learn: 3423.6419695	total: 3.46s	remaining: 10.4s
3000:	learn: 3324.9428784	total: 4.15s	remaining: 9.68s
3500:	learn: 3271.0259509	total: 4.84s	remaining: 8.99s
4000:	learn: 3236.6773432	total: 5.55s	remaining: 8.32s
4500:	learn: 3211.2310169	total: 6.25s	remaining: 7.64s
5000:	learn: 3192.8296974	total: 6.96s	remaining: 6.96s
5500:	learn: 3179.4356660	total: 7.68s	remaining: 6.28s
6000:	learn: 3169.6154798	total: 8.39s	remaining: 5.59s
6500:	learn: 3162.4160780	total: 9.09s	remaining: 4.89s
7000:	learn: 3155.0381910	total: 9.79s	remaining: 4.19s
7500:	learn: 3149.5753116	total: 10.5s	remaining: 3.49s
8000:	learn: 3144.9709340	total:

In [43]:
pre = catr.predict(x_test)

In [44]:
result = []
for i in pre:
    result.append(abs(i))

In [45]:
result

[154.9326861808604,
 25.637456374755857,
 171.47308780321828,
 515.8247346304065,
 116.55590801234439,
 449.6328925417938,
 119.59740814916768,
 301.9520134881759,
 187.78982352567982,
 178.3556681001728,
 82.33466092123035,
 73.03094271185591,
 521.8801508518454,
 87.82406778686709,
 21.45327957663403,
 128.29866974967035,
 134.85284539055147,
 95.79759979613846,
 117.21622370459636,
 225.55319887433234,
 42.83219261673412,
 334.8543278734901,
 122.48425591570003,
 189.60426732378755,
 139.31691396071164,
 22.67617656650191,
 25.25542874642906,
 410.60802930266436,
 136.62943389084194,
 114.64721400011877,
 84.50719547239487,
 1211.3720931690955,
 27.35565338282686,
 9.064871180985392,
 176.3672812181212,
 112.22884929767952,
 211.13152017600623,
 259.1842869130842,
 60.07738169837967,
 260.32407417930153,
 686.555940147734,
 117.53909347635681,
 230.56806828666276,
 47.8587186159516,
 513.6132838695157,
 325.0075111442111,
 253.71683570363075,
 147.6992743703181,
 262.68520773597083,

In [46]:
data.loc[data['reccap'].isnull(),'reccap'] = result

In [50]:
data.to_csv("reccap_predict.csv",index=False)